In [1]:
import os
import sys
from time import sleep
from uuid import uuid4

import GCode
import GRBL
import numpy as np

# Laser Shading 

# Code:

In [2]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    program.M5()
    return program


def end():
    program = GCode.GCode()
    program.M5()
    program.G28()
    return program


In [3]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 2 * BlockHeight  # mm


In [4]:
def fade_line(X, feed=300, n=10):
    prog = GCode.GCode()
    prog.M5()
    powers = np.linspace(0, 255, n, endpoint=True, dtype=np.uint8)[1:]
    Ys = np.linspace(0, BlockHeight, n, endpoint=True)
    prog.G0(X=X, Y=Ys[0], F=feed)
    for i, (power, Y) in enumerate(zip(powers, Ys[1:])):
        prog.M4(S=power)
        prog.G1(X=X, Y=Y, F=feed)
        
        
    prog.M5()
    return prog

In [42]:
X=5
n=20

prog = GCode.GCode()
prog+=init()
for n in [20, 10]:
    for feed in [300, 200, 150, 100, 50]:
        for spacing in [0, 0.1, 0.2, 0.5, 1, 2, 4, 5]:
            X+=spacing
            prog+=fade_line(X, feed=feed, n=n)
        X+=5
prog+=end()

In [44]:
import configparser

In [45]:
config = configparser.ConfigParser()
config["GRBL"]=dict()
config["GRBL"]["port"]="/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

In [46]:
prog.machine=cnc

In [47]:
prog.run()